In [2]:
import json

text = []
f = open('twitter.json')
data = json.load(f)
user = set()
text = []
for obj in data:
    text.append(obj['text'])
    if obj['author'] in user: continue
    text.append(obj['raw_description'])
    if obj['raw_description'] != obj['description']:
        text.append(obj['description'])

len(text)

23897

In [3]:
from tokenizer_digital_language.custom_tokenizer import SpacyCustomTokenizer, get_progressbar

_text = []
vocabulary = set()
nlp = SpacyCustomTokenizer()
bar = get_progressbar(len(text))
bar.start()
for i, t in enumerate(text):
    _text.append(set())
    for token in nlp(t):
        if token.is_stop: continue
        word = token.text.lower() if token.lemma is None else token.lemma.lower()
        vocabulary.add(word)
        _text[-1].add(word)
    bar.update(i + 1)
bar.finish()

/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[#########################################################################]100%


In [4]:
chars = sorted(list(vocabulary))
char_to_num = dict((c, i) for i , c in enumerate(chars))

print(len(chars))

31321


In [5]:
vector_text = []
y_vector = []
bar = get_progressbar(len(text))
bar.start()
for i, t in enumerate(_text):
    y_vector.append([0] * len(_text))
    y_vector[-1][i] = 1
    vector_text.append([0] * len(vocabulary))
    for word in t:
        vector_text[-1][char_to_num[word]] = 1

    bar.update(i + 1)
bar.finish()

[#########################################################################]100%


In [6]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout 

In [7]:
# model = Sequential()
# model.add(Embedding(len(vocabulary), 20))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='sigmoid'))
import tensorflow as tf

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dense_repo):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.dense = tf.keras.layers.Dense(dense_repo)

  def call(self, inputs, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    x = self.dense(x, training=training)
    return x

model = MyModel(len(vocabulary), 20, 1)

model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-30 22:17:17.509659: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-30 22:17:17.510091: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from tensorflow.keras.utils import Sequence
import numpy as np   
from keras.utils import np_utils


class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)

train_gen = DataGenerator(vector_text, y_vector, 32)

In [14]:
for input_example_batch, target_example_batch in train_gen:
    
    example_batch_predictions = model(input_example_batch)
    a= model.predict(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    raise KeyError

In [ ]:
example_batch_predictions

<tf.Tensor: shape=(32, 31321, 1), dtype=float32, numpy=
array([[[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]],

       [[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]],

       [[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]],

       ...,

       [[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]],

       [[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]],

       [[0.4862863],
        [0.509598 ],
        [0.509598 ],
        ...,
        [0.509598 ],
        [0.509598 ],
        [0.509598 ]]], dtype=float32)>